# <center> Data Analysis Project<br><br>Курс "Анализ данных в экономике и финансах"<br><br>Построение дашборда с дополнительными графиками</center>

In [1]:
import os
import io
import json
import base64
from os import path
import numpy as np
import pandas as pd

In [2]:
from getpass import getpass
from mysql.connector import connect, Error

In [3]:
# from datetime import date
import dash
# from dash import html
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.colors
from PIL import ImageColor

<ipython-input-3-f723b5a8bd18>:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-3-f723b5a8bd18>:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [4]:
import urllib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image, ImageDraw

Подключаемся к базе данных **kinopoisk_movies**.

In [5]:
conn_data = connect(host="localhost",
                    port='3306',
                    user='root',
                    password='Kinopoisk231.',
                    database='kinopoisk_movies'
                   )
cur_data = conn_data.cursor()

Вспомогательные функции для построение графиков:
1. **getValues** - получение уникальных значений стоблца таблицы (в т.ч. содержащей несколько значений, разделенных запятой в одной ячейке).
2. **get_color** - получение цветовых кодов из градиентной colorscale для оформления графиков.
3. **get_continuous_color** - вспомогательная функция для предыдущей (**`get_color`**), обрабатывает непосредственно градиент colorscale.
4. **pil_to_b64** - обработка картинок для корректного отображения в HTML.

In [6]:
def getValues(df, column, unique_values, temp_list):
    gen_list_temp = [i.split(', ') for i in df[column]]
    gen_list = []

    for i in gen_list_temp:
        gen_list += i
    
    temp_list.append({'label': 'All', 'value': ''})
    
    if len(unique_values) == 0:
        unique_values.extend(sorted(list(set(gen_list))))
    if len(temp_list) == 1:
        for i in sorted(list(set(gen_list))):
            gen_dict = {}
            gen_dict['label'] = i.title()
            gen_dict['value'] = i
            temp_list.append(gen_dict)

def get_color(colorscale_name, loc):
    from _plotly_utils.basevalidators import ColorscaleValidator
    # first parameter: Name of the property being validated
    # second parameter: a string, doesn't really matter in our use case
    cv = ColorscaleValidator("colorscale", "")
    # colorscale will be a list of lists: [[loc1, "rgb1"], [loc2, "rgb2"], ...] 
    colorscale = cv.validate_coerce(colorscale_name)
    
    if hasattr(loc, "__iter__"):
        return [get_continuous_color(colorscale, x) for x in loc]
    return get_continuous_color(colorscale, loc)

def get_continuous_color(colorscale, intermed):
    if len(colorscale) < 1:
        raise ValueError("colorscale must have at least one color")

    hex_to_rgb = lambda c: "rgb" + str(ImageColor.getcolor(c, "RGB"))

    if intermed <= 0 or len(colorscale) == 1:
        c = colorscale[0][1]
        return c if c[0] != "#" else hex_to_rgb(c)
    if intermed >= 1:
        c = colorscale[-1][1]
        return c if c[0] != "#" else hex_to_rgb(c)

    for cutoff, color in colorscale:
        if intermed > cutoff:
            low_cutoff, low_color = cutoff, color
        else:
            high_cutoff, high_color = cutoff, color
            break

    if (low_color[0] == "#") or (high_color[0] == "#"):
        low_color = hex_to_rgb(low_color)
        high_color = hex_to_rgb(high_color)

    return plotly.colors.find_intermediate_color(
        lowcolor=low_color,
        highcolor=high_color,
        intermed=((intermed - low_cutoff) / (high_cutoff - low_cutoff)),
        colortype="rgb",
    )

def pil_to_b64(im, enc_format="png", **kwargs):

    buff = io.BytesIO()
    im.save(buff, format=enc_format, **kwargs)
    encoded = base64.b64encode(buff.getvalue()).decode("utf-8")

    return encoded

### 🔎 **SQL-запрос: Map**
Запрос выгрузки данных из таблицы со странами с бинарными значениями принадлежности продакшена фильма к каждой из стран: сумма фильмов, относящихся к каждой из стран относительно года выпуска фильма.

In [7]:
cur_data.execute('''CREATE TEMPORARY TABLE temp_table
                    SELECT COLUMN_NAME as column_names
                    FROM INFORMATION_SCHEMA.columns
                    WHERE TABLE_SCHEMA = 'kinopoisk_movies' 
                          AND TABLE_NAME = 'countries'
                          AND COLUMN_NAME NOT LIKE 'kinopoiskId';''') # Создание временной таблицы, занесение в неё наименований колонок таблиц с бинарными признаками (жанры, страны, дистрибьютеры)
conn_data.commit()

cur_data.execute('''UPDATE temp_table SET column_names=CONCAT('SUM(', column_names, ')');''') # Добавление к каждой строке таблицы с названиями колонок SUM( и )
conn_data.commit()

cur_data.execute('''SET session group_concat_max_len = 4096;''') # Исправляем системную переменную - ограничение по максимальному количеству элементов в групповом объединении
conn_data.commit()        
cur_data.execute('''CREATE TEMPORARY TABLE temp_table_q
                    SELECT GROUP_CONCAT(DISTINCT column_names SEPARATOR ', ') as query_sum_part
                    FROM temp_table;''') # Создание второй временной таблицы, которая содержит все SUM(...), разделенные через ", "
conn_data.commit()

cur_data.execute('''UPDATE temp_table_q set query_sum_part=concat('SELECT year, ', query_sum_part, 'FROM movies_upd mu LEFT JOIN countries ON mu.kinopoiskId = countries.kinopoiskId GROUP BY mu.year');''')
conn_data.commit() # Обновление второй временной таблицы, добавление выноски года, а также соединение с главной таблицей

cur_data.execute('''ALTER TABLE temp_table_q
                    ADD COLUMN id INT NOT NULL;''') # Добавляем автоматически заполняемую колонку с уникальными идентификаторами записей
conn_data.commit()

cur_data.execute('''SELECT query_sum_part INTO @sql1 FROM temp_table_q WHERE id = 0;''') # Выбираем нужную ячейку, используя уник. идентификатор
conn_data.commit()
cur_data.execute('''PREPARE sql_query FROM @sql1;''') # Переносим его в подготавливаемый запрос
conn_data.commit()
map_df = pd.read_sql('''EXECUTE sql_query;''', con=conn_data) # Запускаем подготавливаемый запрос

In [8]:
map_df.head()

,year,SUM(Argentina),SUM(Armenia),SUM(Australia),SUM(Austria),SUM(Belarus),SUM(Belgium),SUM(Brazil),SUM(Bulgaria),SUM(Cambodia),...,SUM(Thailand),SUM(the_USSR),SUM(Tunisia),SUM(Turkey),SUM(UAE),SUM(Ukraine),SUM(United_Kingdom),SUM(Uruguay),SUM(USA),SUM(Venezuela)
0,1999,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,0.0
1,2002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
3,2001,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
4,1975,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


Преобразование полученной таблицы: переименование колонок для удобства, а также транспонирование таблицы (чтобы страны оказались в отдельном столбце, а каждый год - каждая колонка).

In [9]:
new_columns = [map_df.columns.to_list()[0]] + [i.split('SUM(')[1].split(')')[0] for i in map_df.columns.to_list()[1::]]

In [10]:
d = {}

for i in range(len(map_df.columns.to_list())):
    if new_columns[i] in map_df.columns.to_list()[i]:
        d[map_df.columns.to_list()[i]] = new_columns[i]

In [11]:
map_df = map_df.rename(columns=d)
map_df = map_df.set_index(['year'])
map_df = map_df.T.reset_index().rename(columns={'index': 'countries'})

Добавим трёхбуквенный код - обозначение стран.

In [12]:
country_codes = pd.read_excel('country_codes.xlsx')

In [13]:
codes_list = []

for i in map_df['countries'].to_list():
    a = country_codes[country_codes['Country'] == i.replace('_', ' ')]['Alpha-3 code'].values
    if len(a) > 0:
        codes_list.append(a[0])
    else:
        if i == 'Czech':
            codes_list.append('CZE')
        elif i == 'Iran':
            codes_list.append('IRN')
        elif i == 'Korea_North':
            codes_list.append('IRN')
        elif i == 'Korea_South':
            codes_list.append('KOR')
        elif i == 'Macedonia':
            codes_list.append('MKD')
        elif i == 'the_USSR':
            codes_list.append('RUS')
        elif i == 'UAE':
            codes_list.append('ARE')
        elif i == 'USA':
            codes_list.append('USA')
        else:
            codes_list.append(' ')

In [14]:
map_df['countries_codes'] = codes_list

Сразу обозначим маркеры для графика в RangeSlider.

In [15]:
map_marks = {}

for i in map_df.columns.to_list()[1:-1]:
    td = {}
    td['label'] = str(i)
    td['style'] = {'transform': 'rotate(-90deg)', '.rc-slider-step': {'background-color': 'red'}}
    map_marks[i] = td

In [16]:
map_df.head()

year,countries,1999,2002,1994,2001,1975,1960,1990,1979,1976,...,2019,2006,2017,2015,2016,2008,2009,2020,2018,countries_codes
0,Argentina,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,ARG
1,Armenia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,ARM
2,Australia,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,5.0,8.0,9.0,0.0,0.0,3.0,5.0,AUS
3,Austria,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,4.0,2.0,0.0,0.0,0.0,0.0,AUT
4,Belarus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,BLR


### 🔎 **SQL-запрос: Pie**
Запрос выгрузки данных из таблицы с дистрибьютерами с бинарными значениями принадлежности каждого фильма к определенной компании: сумма фильмов, относящихся к каждому из дистрибьютеров относительно года выпуска фильма.

In [17]:
cur_data.execute('''CREATE TEMPORARY TABLE temp_table_d
                    SELECT COLUMN_NAME as column_names
                    FROM INFORMATION_SCHEMA.columns
                    WHERE TABLE_SCHEMA = 'kinopoisk_movies' 
                          AND TABLE_NAME = 'distributors'
                          AND COLUMN_NAME NOT LIKE 'kinopoiskId';''')
conn_data.commit()

cur_data.execute('''UPDATE temp_table_d SET column_names=CONCAT('SUM(', column_names, ')');''')
conn_data.commit()

cur_data.execute('''SET session group_concat_max_len = 4096;''')
conn_data.commit()        
cur_data.execute('''CREATE TEMPORARY TABLE temp_table_d1
                    SELECT GROUP_CONCAT(DISTINCT column_names SEPARATOR ', ') as query_sum_part
                    FROM temp_table_d;''')
conn_data.commit()

cur_data.execute('''UPDATE temp_table_d1 set query_sum_part=concat('SELECT year, ', query_sum_part, 'FROM movies_upd mu LEFT JOIN distributors ON mu.kinopoiskId = distributors.kinopoiskId GROUP BY mu.year');''')
conn_data.commit()

cur_data.execute('''ALTER TABLE temp_table_d1
                    ADD COLUMN id INT NOT NULL;''')
conn_data.commit()

cur_data.execute('''SELECT query_sum_part INTO @sql1 FROM temp_table_d1 WHERE id = 0;''')
conn_data.commit()
cur_data.execute('''PREPARE sql_query FROM @sql1;''')
conn_data.commit()
distr_df = pd.read_sql('''EXECUTE sql_query;''', con=conn_data)

In [18]:
distr_df.head()

,year,SUM(20th_Century_Fox_CIS),SUM(25th_Floor_Film),SUM(A_Company),SUM(A_One_Films),SUM(AKM),SUM(All_Media),SUM(Arena),SUM(Argus_SV),SUM(Arthouse),...,SUM(Ten_Letters),SUM(The_art_of_cinema),SUM(Top_Film_Distribution),SUM(UMS_Film),SUM(UPI),SUM(Utopia_Pictures),SUM(Volga),SUM(WDSSPR),SUM(West),SUM(White_Nights)
0,1999,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1975,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Аналогично странам, преобразование полученной таблицы: переименование колонок для удобства, а также транспонирование таблицы (чтобы дистрибьютеры оказались в отдельном столбце, а каждый год - каждая колонка).

In [19]:
new_columns = [distr_df.columns.to_list()[0]] + [i.split('SUM(')[1].split(')')[0] for i in distr_df.columns.to_list()[1::]]

In [20]:
d = {}

for i in range(len(distr_df.columns.to_list())):
    if new_columns[i] in distr_df.columns.to_list()[i]:
        d[distr_df.columns.to_list()[i]] = new_columns[i]

In [21]:
distr_df = distr_df.rename(columns=d)
distr_df = distr_df.set_index(['year'])
distr_df = distr_df.T.reset_index().rename(columns={'index': 'distributors'})

In [22]:
distr_df.head()

year,distributors,1999,2002,1994,2001,1975,1960,1990,1979,1976,...,2014,2019,2006,2017,2015,2016,2008,2009,2020,2018
0,20th_Century_Fox_CIS,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,24.0,11.0,0.0,13.0,22.0,14.0,0.0,0.0,1.0,10.0
1,25th_Floor_Film,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
2,A_Company,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,A_One_Films,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,3.0,3.0,1.0,0.0,0.0,0.0,2.0
4,AKM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0


Временная табличка с суммой фильмов по каждому дистрибьютеру за весь представленный в табличке период.

In [23]:
temp_df_pie = pd.DataFrame({'distributors': distr_df['distributors'].to_list(), 
                            'sum': distr_df[distr_df.columns.to_list()[1::]].sum(axis=1)})

Отберем ТОП-10 дистрибьютеров по самым большим количествам распространяемых фильмов.

In [24]:
temp_df_pie = temp_df_pie.sort_values(by=['sum'], ascending=False, ignore_index=True).loc[:9]
temp_df_pie = temp_df_pie.append({'distributors': 'Others', 'sum': int(sum(distr_df[distr_df.columns.to_list()[1::]].sum(axis=1).to_list())) - int(sum(temp_df_pie['sum'].to_list()))},
                                 ignore_index=True)

Добавим в табличку цвета для каждого дистрибьютера.

In [25]:
temp_df_pie['colors'] = get_color("peach", np.linspace(0, 1, temp_df_pie.shape[0]))

Обозначим маркеры для графика в RangeSlider.

In [26]:
map_marks1 = {}
l1 = sorted(distr_df.columns.to_list()[1::])

n = 0
for i in range(len(l1)):
    if (l1[i] - n) >= 10 or l1[len(l1) - 1] == l1[i]:
        td = {}
        td['label'] = str(l1[i])
        td['style'] = {'transform': 'rotate(-90deg)'}
        map_marks1[l1[i]] = td
        n = l1[i]
        

### 🌀 **WordCloud**
Посмотрим на наиболее часто встречающиеся слова из описания фильмов.

In [27]:
%matplotlib inline

from wordcloud import WordCloud
from collections import defaultdict

**Note:** Обработка всех описаний происходит в течение ~40-50 минут. Чтобы при каждом запуске кода не приходилось долго ждать, лемматизация текста произошла один раз, полученный массив данных записан в несжатом .npz-файле.

In [28]:
# import nltk
# from nltk.corpus import stopwords
# from pymystem3 import Mystem
# from string import punctuation

Формируем стоп-лист слов, пишем функцию для токенизации и лемматизации текста.

In [29]:
# nltk.download("stopwords")

# # Create lemmatizer and stopwords list
# mystem = Mystem() 
# russian_stopwords = stopwords.words("russian")

# #Preprocess function
# def preprocess_text(text):
#     tokens = mystem.lemmatize(text.lower())
#     tokens = [token for token in tokens if token not in russian_stopwords\
#               and token != " " \
#               and token.strip() not in punctuation]
    
#     text = " ".join(tokens)
    
#     return text

#Examples    
# preprocess_text("Ну что сказать, я вижу кто-то наступил на грабли, Ты разочаровал меня, ты был натравлен.")

Добавляем ещё немного слов в стоп-лист.

In [30]:
# russian_stopwords.append('который')
# russian_stopwords.append('свой')
# russian_stopwords.append('каждый')
# russian_stopwords.append('весь')
# russian_stopwords.append('это')
# russian_stopwords.append('"')
# russian_stopwords.append('-')

In [31]:
# word_bubble = []

Считываем данные таблицы.

In [32]:
# mov_upd_df = pd.read_sql('''SELECT * FROM movies_upd''',
#                          con=conn_data)

Препроцессим описания фильмов.

In [33]:
# %%time

# for i in mov_upd_df.description:
#     lmmd = preprocess_text(i)
#     word_bubble.append(lmmd)

Сохраняем список полученных лемматизированных слов в .npz.

In [34]:
# np.savez("word_bubble", word_bubble)

Считываем список предобработанных слов, формируем словарь, содержащий слово и количество его вхождений в описания. Рисуем облако слов в картинке-маске 🦇.

In [35]:
word_bubble1 = np.load('word_bubble.npz')
word_bubble1 = word_bubble1['arr_0']
word_bubble1 = word_bubble1.tolist()

all_w = defaultdict(int)

for deff in word_bubble1:
    for word in deff.split(' '):
        all_w[word.strip()] += 1

d = path.dirname(__file__) if "__file__" in locals() else os.getcwd()
camera_mask = np.array(Image.open(path.join(d, "Б2.png")))    

cloud = WordCloud(font_path='Montserrat-Bold.ttf', max_words=2000,
                  mask=camera_mask, 
                  background_color='white', # contour_width=3,
                  colormap='OrRd').generate_from_frequencies(all_w)

cloud.to_file("first_review.png")

### 📊 **Dashboard**
Формируем графики для дашборда, дефолтные и пользовательские запросы в БД по каждому из них.

In [ ]:
# ---------------------------------------------------------------
# Выбираем уникальные значения жанров из главной таблицы, 
# формируем доступные элементы для выпадающего списка

genres_df = pd.read_sql('''SELECT genres FROM kinopoisk_movies.movies_upd mu''', con=conn_data)
genres_list_main = []
getValues(genres_df, 'genres', [None], genres_list_main)
# print(genres_list_main)

# ---------------------------------------------------------------
# Выбираем уникальные значения возрастных ограничений из главной таблицы, 
# формируем доступные элементы для выпадающего списка,
# подбираем цветовую палетку для каждого распределения

unique_rating_age_limits = pd.read_sql('''SELECT DISTINCT ratingAgeLimits
                                          FROM kinopoisk_movies.movies_upd mu''', con=conn_data)
unique_rating_age_limits = unique_rating_age_limits.sort_values(by=['ratingAgeLimits'], ascending=False)
unique_rating_age_limits['colors'] = get_color("peach", np.linspace(0, 1, unique_rating_age_limits.shape[0]))
distplot_default_values = unique_rating_age_limits['ratingAgeLimits'].to_list()
ratingagelimits_list_main = []
for u in distplot_default_values:
    d = {}
    d['label'] = u
    d['value'] = u
    ratingagelimits_list_main.append(d)
# print(ratingagelimits_list_main)


# ---------------------------------------------------------------
# Функция для построения карты с интенсивностью окраски 
# в зависимости от количества фильмов за период по странам

def MapGraph(map_df, sums):
    fig = go.Figure(data=go.Choropleth(
        locations = map_df['countries_codes'],
        z = sums,
        hovertemplate = '<b>%{text}</b>'
                        '<br>Количество фильмов: %{z:.0f}<extra></extra>',
        text = ['Страна: {}'.format(i) for i in map_df['countries']],
        colorscale = 'Peach',
        autocolorscale=False,
        reversescale=True,
        marker_line_color='white',
        marker_line_width=0.5,
        colorbar_title = 'Количество<br>фильмов<br>',
    ))

    fig.update_layout(
        height=420,
        margin=dict(l=0,
                    r=0,
                    b=20,
                    t=30),
        geo=dict(
            showframe=False,
            showcoastlines=False
        )
    )

    return fig


# ---------------------------------------------------------------
# Функция для построения круговой диаграммы доли 
# дистрибьютеров в распростанении фильмов

def PieChart(pie_df):
    fig = go.Figure(data=[go.Pie(
        labels=[i.replace('_', ' ') for i in pie_df['distributors'].to_list()], 
        values=pie_df['sum'],
        hole=.5,
        marker_colors=pie_df['colors']
    )])

    fig.update_traces( 
        textfont_size=13,
        marker=dict(line=dict(color='white', width=2.5))
    )
    
    fig.update_layout(
        height=420,
        margin=dict(l=0,
                    r=0,
                    b=0,
                    t=50),
        legend_title_text='Дистрибьютеры<br>'
    )

    return fig


# ---------------------------------------------------------------
# Функция для построения ТОП-5 фильмов по:
# бюджету, российских и мировых кассовых сборов

def BarPlots5Graph():
    bar_plot_df_1 = pd.read_sql('''SELECT * FROM (SELECT 
                                                    replace(name, ': ', ':<br>') name, 
                                                    budget_USD_d, 
                                                    worldBO_USD_d, 
                                                    rusBO_USD_d 
                                                  FROM kinopoisk_movies.movies_upd
                                                  ORDER BY budget_USD_d DESC
                                                  LIMIT 5) table_b
                                   ORDER BY budget_USD_d'''
                                , con = conn_data)

    bar_plot_df_2 = pd.read_sql('''SELECT * FROM (SELECT 
                                                    replace(name, ': ', ':<br>') name, 
                                                    worldBO_USD_d
                                                  FROM kinopoisk_movies.movies_upd
                                                  ORDER BY worldBO_USD_d DESC
                                                  LIMIT 5) table_bow
                                   ORDER BY worldBO_USD_d'''
                                , con = conn_data)

    bar_plot_df_3 = pd.read_sql('''SELECT * FROM (SELECT 
                                                    replace(name, ': ', ':<br>') name, 
                                                    rusBO_USD_d 
                                                  FROM kinopoisk_movies.movies_upd
                                                  ORDER BY rusBO_USD_d DESC
                                                  LIMIT 5) table_bor
                                   ORDER BY rusBO_USD_d'''
                                , con = conn_data)

    fig = make_subplots(
        rows=2, cols=4,
        specs=[[{"rowspan": 2, "colspan": 2}, {}, {}, {}],
               [None, None, {}, {}]])


    fig.add_trace(go.Bar(
        x=bar_plot_df_1['budget_USD_d'], 
        y=bar_plot_df_1['name'], 
        orientation = 'h', 
        name="Бюджет",
        marker=dict(color='#ED5D47'),
        hovertemplate = '<b>Фильм: %{y}</b>' + '<br>Бюджет: %{x}'), 
                  row=1, col=1)

    fig.add_trace(go.Scatter(
        x=bar_plot_df_1['budget_USD_d'] + 80000000, 
        y=bar_plot_df_1['name'], 
        orientation = 'h', mode='markers+text', 
        name="Кассовые сборы (мир)",
        marker=dict(color='#FEC200', size=10),
        hovertemplate = '<b>Фильм: %{y}</b>' + '<br>Кассовые сборы (мир): %{text}',
        text=['{}M'.format(i) for i in round(bar_plot_df_1['worldBO_USD_d'] / 1000000,1)],
        textposition="bottom center"), 
                  row=1, col=1)

    fig.add_trace(go.Scatter(
        x=bar_plot_df_1['budget_USD_d']+220000000, 
        y=bar_plot_df_1['name'], 
        orientation = 'h', mode='markers+text',
        name="Кассовые сборы (Россия)",
        marker=dict(color='#8DD8FA', size=10),
        hovertemplate = '<b>Фильм: %{y}</b>' + '<br>Кассовые сборы (Россия): %{text}',
        text=['{}M'.format(i) for i in round(bar_plot_df_1['rusBO_USD_d'] / 1000000, 1)],
        textposition="bottom center"), 
                  row=1, col=1)

    fig.add_trace(go.Bar(
        x=bar_plot_df_2['worldBO_USD_d'], 
        y=bar_plot_df_2['name'], 
        orientation = 'h', 
        name="Кассовые сборы (мир)",
        marker=dict(color='#F59E72'),
        hovertemplate = '<b>Фильм: %{y}</b>' + '<br>Бюджет: %{x}'), 
                  row=2, col=4)

    fig.add_trace(go.Bar(
        x=bar_plot_df_3['rusBO_USD_d'], 
        y=bar_plot_df_3['name'], 
        orientation = 'h', 
        name="Кассовые сборы (Россия)",
        marker=dict(color='#FBD3B2'),
        hovertemplate = '<b>Фильм: %{y}</b>' + '<br>Бюджет: %{x}'), 
                  row=1, col=4)

    fig.update_layout(template = 'plotly_white',
                      margin=dict(
                          pad=10,
                          l=0,
                          r=0,
                          b=0,
                          t=10
                      ),
                      legend=dict(
                          orientation="v",
                          yanchor="top",
                          y=0.65,
                          xanchor="right",
                          x=1.32
                      ),
                      height=400
                     )

    return fig


# ---------------------------------------------------------------
# Дефолтный запрос и функция диаграммы рассеяния 
# зависимости рейтинга пользователей КиноПоиска от размера бюджета 
# (размер маркеров зависит от величины российских кассовых сборов)

scatter_plot_df = pd.read_sql('''SELECT name, budget_USD_d, ratingKinopoisk, rusBO_USD_d, genres 
                                 FROM kinopoisk_movies.movies_upd mu
                                 WHERE genres LIKE "%{}%"'''.format(''), 
                              con=conn_data)

def ScatterPlotGraph(scatter_plot_df):
    fig = go.Figure(
        data=[go.Scatter(
            x=scatter_plot_df['budget_USD_d'], 
            y=scatter_plot_df['ratingKinopoisk'],
            mode='markers',
            marker=dict(
                color=np.arange(120, 26355, 15),
                size=scatter_plot_df['rusBO_USD_d'] / 1000000,
                colorscale='peach'),
            hovertemplate = '<b>%{text}</b>' +
                            '<br>Рейтинг КиноПоиск: %{y:.2f}<extra></extra>' +
                            '<br>Бюджет: %{x:$,}',
            text = ['Фильм: {}'.format(i) for i in scatter_plot_df['name']]
        )
             ])
    
    fig.update_layout(
        xaxis_range=[-50000000, 450000000],
        yaxis_range=[0, 10],
        template='plotly_white',
        margin=dict(l=0,
                    r=0,
                    b=0,
                    t=50),
        height=420
    )

    fig.update_xaxes(showline=True, linewidth=2, title_text='<i>Бюджет фильма</i>')
    fig.update_yaxes(showline=True, linewidth=2, title_text='<i>Рейтинг КиноПоиск</i>')
    
    return fig


# ---------------------------------------------------------------
# Дефолтный запрос и функция диаграммы распределения 
# прибыли фильма от его возрастных ограничений

distplot_plot_df = pd.read_sql('''SELECT name, worldBO_USD_d - budget_USD_d AS revenue, ratingAgeLimits
                                  FROM kinopoisk_movies.movies_upd mu
                                  WHERE ratingAgeLimits REGEXP "{}"'''.format('|'.join([str(i) for i in distplot_default_values])),
                               con=conn_data)

def DistplotGraph(distplot_plot_df):
    fig1 = make_subplots(rows=2, cols=1, specs=[[{"secondary_y": False}],
                                               [{"secondary_y": False}]],
                        row_heights=[0.25, 0.75],
                        vertical_spacing=0.04)

    for i in list(distplot_plot_df['ratingAgeLimits'].unique()):
        fig1.add_trace(
            go.Box(x=distplot_plot_df[distplot_plot_df['ratingAgeLimits'] == i]['revenue'],
                   name='<i>' + str(i) + '</i>',
                   notched=True,
                   marker=dict(color=unique_rating_age_limits[unique_rating_age_limits['ratingAgeLimits'] == i]['colors'].values[0]),
                   opacity=0.85),
            row=1, col=1,
            secondary_y=False
        )

        fig1.add_trace(
            go.Histogram(
                x=distplot_plot_df[distplot_plot_df['ratingAgeLimits'] == i]['revenue'],
                name='<i>' + 'Возраст:' + str(i) + '</i>',
                marker=dict(color=unique_rating_age_limits[unique_rating_age_limits['ratingAgeLimits'] == i]['colors'].values[0]),
                opacity=0.7),
            row=2, col=1,
            secondary_y=False
        )

    fig1.update_layout(
        showlegend=False,
        template='plotly_white'
    )

    fig1["layout"]["xaxis1"].update((dict(range=[-165000000, 2500000000])))
    fig1["layout"]["yaxis1"].update(dict(title='<i>Возраст</i>'))
    fig1["layout"]["xaxis2"].update(dict(title='<i>Прибыль</i>', range=[-165000000, 2500000000]))
    fig1["layout"]["yaxis2"].update(dict(title='<i>Количество</i>'))

    fig1.update_layout(
        barmode='overlay',
        margin=dict(l=0,
                    r=0,
                    b=0,
                    t=50),
#         width=680,
        height=420
    )

    return fig1

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# ---------------------------------------------------------------
# Запросы для ТОП-3 персонала

b3_directors = pd.read_sql('''SELECT staff_id, nameRu, nameEn, birthday, birthplace, hasAwards, profession, name
                              FROM movies_upd mu
                                  INNER JOIN staff_upd on mu.director_id = staff_upd.staff_id
                              ORDER BY worldBO_USD_d DESC
                              LIMIT 4;''', con=conn_data)
b3_directors = b3_directors.drop_duplicates(ignore_index=True)
b3_directors = b3_directors.loc[:2]

b3_actors = pd.read_sql('''SELECT staff_id, nameRu, nameEn, birthday, birthplace, hasAwards, profession
                           FROM movies_upd mu
                               INNER join staff_upd on mu.actor_id = staff_upd.staff_id
                           GROUP BY actor_id
                           HAVING COUNT(actor_id)
                           ORDER BY count(actor_id) DESC
                           LIMIT 4;''', con=conn_data)
b3_actors = b3_actors.drop_duplicates(ignore_index=True)
b3_actors = b3_actors.loc[:2]

b3_producers = pd.read_sql('''SELECT staff_id, nameRu, nameEn, birthday, birthplace, hasAwards, profession
                              FROM movies_upd mu
                                  INNER join staff_upd on mu.producer_id = staff_upd.staff_id
                              ORDER BY budget_USD_d DESC
                              LIMIT 4;''', con=conn_data)
b3_producers = b3_producers.drop_duplicates(ignore_index=True)
b3_producers = b3_producers.loc[:2]

# ---------------------------------------------------------------
# Извлечение фото персоны из КиноПоиска и его препросессинг (изменение размера, круговая обрезка)

def getImage(df, new_size):
    for i in df['staff_id']:
        img = Image.open(
            urllib.request.urlopen(
                'https://kinopoiskapiunofficial.tech/images/actor_posters/kp/' + str(i) + '.jpg')
        ).convert("RGB")
        w, h = img.size
        # -- ||

        t = min(w, h)
        w1 = (w - t) / 2
        w2 = (w + t) / 2
        h1 = (h - t) / 2
        h2 = (h + t) / 2

        img = img.crop((w1, h1, w2, h2))
        newsize = (new_size, new_size)
        img = img.resize(newsize)

        w, h = img.size
        alpha = Image.new('L', img.size, 0)
        draw = ImageDraw.Draw(alpha)
        draw.pieslice([0, 0, h, h], 0, 360, fill=255)

        npImage=np.array(img)
        npAlpha=np.array(alpha)

        npImage = np.dstack((npImage, npAlpha))

        return Image.fromarray(npImage)

    
# ---------------------------------------------------------------
# Сборка дашборда:

app.layout = html.Div(children=[
    html.Div([
        html.Div([
            html.H3(children='Распределение количества произведенных', style={'line-height': '.8',
                                                                              'margin-bottom': '0.05rem',
                                                                              'padding-top': '18px',
                                                                              'padding-left': '20px'}),
            html.H3(children='фильмов по странам', style={'line-height': '.8',
                                                          'margin-bottom': '0.15rem',
                                                          'padding-left': '20px',
                                                          'padding-bottom': '2px'}),
            dcc.Graph(
                id='map',
                figure=MapGraph(map_df, map_df[map_df.columns.to_list()[1:-1]].sum(axis=1))
            ),
            dcc.RangeSlider(
                id='my-range-slider',
                min=min(map_df.columns.to_list()[1:-1]),
                max=max(map_df.columns.to_list()[1:-1]),
                step=1,
                marks=map_marks,
                value=[min(map_df.columns.to_list()[1:-1]), max(map_df.columns.to_list()[1:-1])],
                tooltip={"placement": "bottom", "always_visible": True}
            )
        ], className='six columns', style={'width': '62%',
                                           'padding-bottom': '10px',
                                           'padding-left': '10px',
                                           'border-radius': '10px',
                                           'box-shadow': '0 0 10px rgba(0,0,0,0.25)',
                                           'margin': 5, 'display': 'inline-block'}),
        html.Div([
            html.H3(children='Доли компаний в', style={'line-height': '.8',
                                                       'margin-bottom': '0.05rem',
                                                       'padding-top': '18px',
                                                       'padding-left': '20px'}),
            
            html.H3(children='дистрибуции фильмов', style={'line-height': '.8',
                                                           'margin-bottom': '0.05rem',
                                                           'padding-left': '20px',
                                                           'padding-bottom': '2px'}),
            dcc.Graph(
                id='pie',
                figure=PieChart(temp_df_pie)
            ),
            dcc.RangeSlider(
                id='my-range-slider1',
                min=min(distr_df.columns.to_list()[1::]),
                max=max(distr_df.columns.to_list()[1::]),
                step=1,
                marks=map_marks1,
                value=[min(distr_df.columns.to_list()[1::]), max(distr_df.columns.to_list()[1::])],
                tooltip={"placement": "bottom", "always_visible": True}
            )
        ], className='six columns', style={'width': '36%',
                                           'padding-bottom': '10px',
                                           'padding-left': '10px',
                                           'padding-right': '10px',
                                           'border-radius': '10px',
                                           'box-shadow': '0 0 10px rgba(0,0,0,0.25)',
                                           'margin': 5, 'display': 'inline-block'})
    ], className='row', style={'grid-column-gap': '10px'}),
    html.Div([
        html.Div([
            html.H3(children='Облако слов на основе', style={ 'line-height': '.8',
                                                             'margin-bottom': '0.05rem',
                                                             'padding-top': '20px',
                                                             'padding-left': '20px'}),
            html.H3(children='описания фильмов', style={'line-height': '.8',
                                                        'margin-bottom': '0.05rem',
                                                        'padding-left': '20px',
                                                        'padding-bottom': '60px'}),
            html.Img(
                id='cloud',
                src="data:image/png;base64, " + pil_to_b64(Image.fromarray(np.asarray(Image.open('first_review.png')))),
                style={'width':'99%', 
                       'height': '125%', 
                       'padding-bottom': '50px'}
            )
        ], className='six columns', style={'width': '36%',
                                           'padding-bottom': '10px',
                                           'padding-left': '10px',
                                           'border-radius': '10px',
                                           'box-shadow': '0 0 10px rgba(0,0,0,0.25)',
                                           'margin': 5, 'display': 'inline-block'}),
        html.Div([
            html.H3(children='TOP-5 фильмов по бюджетам и кассовым сборам, USD', style={'line-height': '.8',
                                                                                        'margin-bottom': '0.05rem',
                                                                                        'padding-top': '20px',
                                                                                        'padding-left': '20px',
                                                                                        'padding-bottom': '30px'}),
            dcc.Graph(
                id='top-5',
                figure=BarPlots5Graph()
            )
            
        ], className='six columns', style={'width': '62%',
                                           'padding-bottom': '10px',
                                           'padding-left': '10px',
                                           'padding-left': '10px',
                                           'border-radius': '10px',
                                           'box-shadow': '0 0 10px rgba(0,0,0,0.25)',
                                           'margin': 5, 'display': 'inline-block'})
    ], className='row', style={'grid-column-gap': '10px'}),
    html.Div([
        html.Div([
            html.H3(children='Соотношение бюджета фильма', style={'line-height': '.8',
                                                                  'margin-bottom': '0.05rem',
                                                                  'padding-top': '20px',
                                                                  'padding-left': '20px'}),
            html.H3(children='и рейтинга КиноПоиск', style={'line-height': '.8',
                                                            'margin-bottom': '0.05rem',
                                                            'padding-left': '20px',
                                                            'padding-bottom': '30px'}),
            dcc.Graph(
                id='fig',
                figure=ScatterPlotGraph(scatter_plot_df)
            ),
            html.Br(),
            html.Div(id='output_data'),
            html.Label(['Выберите жанр:'],style={'font-weight': 'bold', "text-align": "center"}),
            dcc.Dropdown(id='my_dropdown',
                         options=genres_list_main,
                         optionHeight=35,
                         value='',
                         searchable=True,
                         placeholder='Please select...',
                         style={'width': '150px',
                                'align-items': 'center'},
                        ),
        ], className='six columns', style={'width': '49%',
                                           'border-radius': '10px',
                                           'padding-bottom': '10px',
                                           'padding-left': '10px',
                                           'padding-right': '10px',
                                           'box-shadow': '0 0 10px rgba(0,0,0,0.25)',
                                           'margin': 5, 'display': 'inline-block'}),
        html.Div([
            html.H3(children='Распределение прибыли относительно', style={'line-height': '.8',
                                                                          'margin-bottom': '0.05rem',
                                                                          'padding-top': '20px',
                                                                          'padding-left': '20px'}),
            html.H3(children='возрастных ограничений', style={'line-height': '.8',
                                                              'margin-bottom': '0.05rem',
                                                              'padding-left': '20px',
                                                              'padding-bottom': '30px'}),
            dcc.Graph(
                id='fig1',
                figure=DistplotGraph(distplot_plot_df)
            ),
            html.Br(),
            html.Div(id='output_data1'),
            html.Label(['Выберите возрастные ограничения:'],style={'font-weight': 'bold', "text-align": "center"}),
            dcc.Dropdown(id='my_dropdown1',
                         options=ratingagelimits_list_main,
                         optionHeight=35,
                         value=distplot_default_values,
                         multi=True,
                         searchable=True,
                         placeholder='Please select...',
                         style={'width': '300px'},
                        ),
        ], className='six columns', style={'width': '49%',
                                           'border-radius': '10px',
                                           'padding-bottom': '10px',
                                           'padding-left': '10px',
                                           'padding-right': '10px',
                                           'box-shadow': '0 0 10px rgba(0,0,0,0.25)',
                                           'margin': 5, 'display': 'inline-block'}),
    ], className='row', style={'grid-column-gap': '0px'}),
    html.Div([
        html.H1(children='ТОП-3 создателей фильмов', style={'font-weight': 'bold',
                                                            'text-align': 'center'}),
        html.Div([
            html.Div([
                html.H5(children='Режиссёры фильмов с самыми крупными мировыми кассовыми сборами:')
            ], className='four columns', style={'padding-left': '20px',
                                                'padding-right': '20px',
                                                'border-radius': '10px',
                                                'box-shadow': '0 0 13px rgba(237, 92, 70, 1)'}),
            html.Div([
                html.H5(children='Актёры, у которых больше всего главных ролей в фильмах:')
            ], className='four columns', style={'padding-left': '20px',
                                                'padding-right': '20px',
                                                'border-radius': '10px',
                                                'box-shadow': '0 0 13px rgba(237, 92, 70, 1)'}),
            html.Div([
                html.H5(children='Продюсеры самых высокобюджетных фильмов:')
            ], className='four columns', style={'padding-left': '20px',
                                                'padding-right': '20px',
                                                'border-radius': '10px',
                                                'box-shadow': '0 0 13px rgba(237, 92, 70, 1)'})
        ], className='row'),
        html.Div([
            html.Div([
                html.Img(
                    id='director1',
                    src="data:image/png;base64, " + pil_to_b64(getImage(b3_directors.loc[[0]], 220))
                )
            ], className='two columns', style={'padding-left': '20px',
                                               'padding-top': '50px'}),
            html.Div([
                html.H2(children=b3_directors['nameRu'][0], style={'font-weight': 'bold',
                                                                   'line-height': '1.2',
                                                                   'margin-bottom': '0.15rem'}),
                html.H6(children='англ. ' + b3_directors['nameEn'][0], style={'line-height': '1',
                                                                              'margin-bottom': '1.75rem'}),
                html.H6(children=b3_directors['profession'][0], style={'font-size': '16px',
                                                                       'font-style': 'italic',
                                                                       'line-height': '1.2',
                                                                       'margin-bottom': '0.1rem'}),
                html.H6(children='Дата рождения: ' + str('.'.join(reversed(b3_directors['birthday'][0].split('-')))),
                        style={'font-size': '16px',
                               'line-height': '1.2',
                               'margin-bottom': '0.1rem'}),
                html.H6(children='Место рождения: ' + str(b3_directors['birthplace'][0]), style={'font-size': '16px',
                                                                                                 'line-height': '1.2'})
            ], className='two columns', style={'padding-top': '7px'}),
            html.Div([
                html.Img(
                    id='actor1',
                    src="data:image/png;base64, " + pil_to_b64(getImage(b3_actors.loc[[0]], 220))
                )
            ], className='two columns', style={'padding-left': '20px',
                                               'padding-top': '50px'}),
            html.Div([
                html.H2(children=b3_actors['nameRu'][0], style={'font-weight': 'bold',
                                                                'line-height': '1.2',
                                                                'margin-bottom': '0.15rem'}),
                html.H6(children='англ. ' + b3_actors['nameEn'][0], style={'line-height': '1',
                                                                           'margin-bottom': '1.75rem'}),
                html.H6(children=b3_actors['profession'][0], style={'font-size': '16px',
                                                                    'font-style': 'italic',
                                                                    'line-height': '1.2',
                                                                    'margin-bottom': '0.1rem'}),
                html.H6(children='Дата рождения: ' + str('.'.join(reversed(b3_actors['birthday'][0].split('-')))),
                        style={'font-size': '16px',
                               'line-height': '1.2',
                               'margin-bottom': '0.1rem'}),
                html.H6(children='Место рождения: ' + str(b3_actors['birthplace'][0]), style={'font-size': '16px',
                                                                                              'line-height': '1.2'})
            ], className='two columns', style={'padding-top': '7px'}),
            html.Div([
                html.Img(
                    id='producer1',
                    src="data:image/png;base64, " + pil_to_b64(getImage(b3_producers.loc[[0]], 220))
                )
            ], className='two columns', style={'padding-left': '20px',
                                               'padding-top': '50px'}),
            html.Div([
                html.H2(children=b3_producers['nameRu'][0], style={'font-weight': 'bold',
                                                                   'line-height': '1.2',
                                                                   'margin-bottom': '0.15rem'}),
                html.H6(children='англ. ' + b3_producers['nameEn'][0], style={'line-height': '1',
                                                                              'margin-bottom': '1.75rem'}),
                html.H6(children=b3_producers['profession'][0], style={'font-size': '16px',
                                                                       'font-style': 'italic',
                                                                       'line-height': '1.2',
                                                                       'margin-bottom': '0.1rem'}),
                html.H6(children='Дата рождения: ' + str('.'.join(reversed(b3_producers['birthday'][0].split('-')))), 
                        style={'font-size': '16px',
                               'line-height': '1.2',
                               'margin-bottom': '0.1rem'}),
                html.H6(children='Место рождения: ' + str(b3_producers['birthplace'][0]), style={'font-size': '16px',
                                                                                                 'line-height': '1.2'})
            ], className='two columns', style={'padding-top': '7px',
                                               'padding-right': '20px'}),
        ], className='row'),
        html.Div([
            # 2-3 directors
            html.Div([
                html.Img(
                    id='director2',
                    src="data:image/png;base64, " + pil_to_b64(getImage(b3_directors.loc[[1]], 100))
                ),
                html.Br(),
                html.H5(children=b3_directors['nameRu'][1], style={'font-weight': 'bold',
                                                                   'line-height': '1.2',
                                                                   'margin-bottom': '0.15rem'}),
                html.H6(children='англ. ' + b3_directors['nameEn'][1], style={'font-size': '14px',
                                                                              'line-height': '1',
                                                                              'margin-bottom': '1.75rem'}),
                html.H6(children=b3_directors['profession'][1], style={'font-size': '14px',
                                                                       'font-style': 'italic',
                                                                       'line-height': '1.2',
                                                                       'margin-bottom': '0.1rem'}),
                html.H6(children='Дата рождения: ' + str('.'.join(reversed(b3_directors['birthday'][1].split('-')))),
                        style={'font-size': '14px',
                               'line-height': '1.2',
                               'margin-bottom': '0.1rem'}),
                html.H6(children='Место рождения: ' + str(b3_directors['birthplace'][1]), style={'font-size': '14px',
                                                                                                 'line-height': '1.2'})
            ], className='six columns', style={'width': '14%',
                                               'padding-left': '20px',
                                               'padding-top': '7px',
                                               'margin': 10, 'display': 'inline-block'
                                              }),
            html.Div([
                html.Img(
                    id='director3',
                    src="data:image/png;base64, " + pil_to_b64(getImage(b3_directors.loc[[2]], 100))
                ),
                html.Br(),
                html.H5(children=b3_directors['nameRu'][2], style={'font-weight': 'bold',
                                                                   'line-height': '1.2',
                                                                   'margin-bottom': '0.15rem'}),
                html.H6(children='англ. ' + b3_directors['nameEn'][2], style={'font-size': '14px',
                                                                              'line-height': '1',
                                                                              'margin-bottom': '1.75rem'}),
                html.H6(children=b3_directors['profession'][2], style={'font-size': '14px',
                                                                       'font-style': 'italic',
                                                                       'line-height': '1.2',
                                                                       'margin-bottom': '0.1rem'}),
                html.H6(children='Дата рождения: ' + str('.'.join(reversed(b3_directors['birthday'][2].split('-')))),
                        style={'font-size': '14px',
                               'line-height': '1.2',
                               'margin-bottom': '0.1rem'}),
                html.H6(children='Место рождения: ' + str(b3_directors['birthplace'][2]), style={'font-size': '14px',
                                                                                                 'line-height': '1.2'})
            ], className='six columns', style={'width': '14%',
                                               'padding-top': '7px',
                                               'margin': 10, 'display': 'inline-block'}),
            # 2-3 actors
            html.Div([
                html.Img(
                    id='actor2',
                    src="data:image/png;base64, " + pil_to_b64(getImage(b3_actors.loc[[1]], 100))
                ),
                html.Br(),
                html.H5(children=b3_actors['nameRu'][1], style={'font-weight': 'bold',
                                                                'line-height': '1.2',
                                                                'margin-bottom': '0.15rem'}),
                html.H6(children='англ. ' + b3_actors['nameEn'][1], style={'font-size': '14px',
                                                                           'line-height': '1',
                                                                           'margin-bottom': '1.75rem'}),
                html.H6(children=b3_actors['profession'][1], style={'font-size': '14px',
                                                                    'font-style': 'italic',
                                                                    'line-height': '1.2',
                                                                    'margin-bottom': '0.1rem'}),
                html.H6(children='Дата рождения: ' + str('.'.join(reversed(b3_actors['birthday'][1].split('-')))),
                        style={'font-size': '14px',
                               'line-height': '1.2',
                               'margin-bottom': '0.1rem'}),
                html.H6(children='Место рождения: ' + str(b3_actors['birthplace'][1]), style={'font-size': '14px',
                                                                                              'line-height': '1.2'})
            ], className='six columns', style={'width': '18%',
                                               'padding-left': '76px',
                                               'padding-top': '7px',
                                               'margin': 10, 'display': 'inline-block'
                                              }),
            html.Div([
                html.Img(
                    id='actor3',
                    src="data:image/png;base64, " + pil_to_b64(getImage(b3_actors.loc[[2]], 100))
                ),
                html.Br(),
                html.H5(children=b3_actors['nameRu'][2], style={'font-weight': 'bold',
                                                                'line-height': '1.2',
                                                                'margin-bottom': '0.15rem'}),
                html.H6(children='англ. ' + b3_actors['nameEn'][2], style={'font-size': '14px',
                                                                           'line-height': '1',
                                                                           'margin-bottom': '1.75rem'}),
                html.H6(children=b3_actors['profession'][2], style={'font-size': '14px',
                                                                    'font-style': 'italic',
                                                                    'line-height': '1.2',
                                                                    'margin-bottom': '0.1rem'}),
                html.H6(children='Дата рождения: ' + str('.'.join(reversed(b3_actors['birthday'][2].split('-')))),
                        style={'font-size': '14px',
                               'line-height': '1.2',
                               'margin-bottom': '0.1rem'}),
                html.H6(children='Место рождения: ' + str(b3_actors['birthplace'][2]), style={'font-size': '14px',
                                                                                              'line-height': '1.2'})
            ], className='six columns', style={'width': '14%',
                                               'padding-top': '7px',
                                               'margin': 10, 'display': 'inline-block'}),
            # 2-3 producers
            html.Div([
                html.Img(
                    id='producer2',
                    src="data:image/png;base64, " + pil_to_b64(getImage(b3_producers.loc[[1]], 100))
                ),
                html.Br(),
                html.H5(children=b3_producers['nameRu'][1], style={'font-weight': 'bold',
                                                                   'line-height': '1.2',
                                                                   'margin-bottom': '0.15rem'}),
                html.H6(children='англ. ' + b3_producers['nameEn'][1], style={'font-size': '14px',
                                                                              'line-height': '1',
                                                                              'margin-bottom': '1.75rem'}),
                html.H6(children=b3_producers['profession'][1], style={'font-size': '14px',
                                                                       'font-style': 'italic',
                                                                       'line-height': '1.2',
                                                                       'margin-bottom': '0.1rem'}),
                html.H6(children='Дата рождения: ' + str('.'.join(reversed(b3_producers['birthday'][1].split('-')))),
                        style={'font-size': '14px',
                               'line-height': '1.2',
                               'margin-bottom': '0.1rem'}),
                html.H6(children='Место рождения: ' + str(b3_producers['birthplace'][1]), style={'font-size': '14px',
                                                                                                 'line-height': '1.2'})
            ], className='six columns', style={'width': '18%',
                                               'padding-left': '72px',
                                               'padding-top': '7px',
                                               'margin': 10, 'display': 'inline-block'
                                              }),
            html.Div([
                html.Img(
                    id='producer3',
                    src="data:image/png;base64, " + pil_to_b64(getImage(b3_producers.loc[[2]], 100))
                ),
                html.Br(),
                html.H5(children=b3_producers['nameRu'][2], style={'font-weight': 'bold',
                                                                   'line-height': '1.2',
                                                                   'margin-bottom': '0.15rem'}),
                html.H6(children='англ. ' + b3_producers['nameEn'][2], style={'font-size': '14px',
                                                                              'line-height': '1',
                                                                              'margin-bottom': '1.75rem'}),
                html.H6(children=b3_producers['profession'][2], style={'font-size': '14px',
                                                                       'font-style': 'italic',
                                                                       'line-height': '1.2',
                                                                       'margin-bottom': '0.1rem'}),
                html.H6(children='Дата рождения: ' + str('.'.join(reversed(b3_producers['birthday'][2].split('-')))),
                        style={'font-size': '14px',
                               'line-height': '1.2',
                               'margin-bottom': '0.1rem'}),
                html.H6(children='Место рождения: ' + str(b3_producers['birthplace'][2]), style={'font-size': '14px',
                                                                                                 'line-height': '1.2'})
            ], className='six columns', style={'width': '14%',
                                               'padding-top': '7px',
                                               'margin': 10, 'display': 'inline-block'}),
        ], className='row'),
        ], className='row', style={'padding-top': '20px',
                                   'margin': 5, 'display': 'inline-block'}
    )
])

# Connecting the Dropdown values to the graph

# ---------------------------------------------------------------
# Пользовательское обновление карты:
@app.callback(
    Output(component_id='map', component_property='figure'),
    [Input(component_id='my-range-slider', component_property='value')]
)

def MapGraph_upd(value):
    new_values_list_map = []
    for i in map_df.columns.to_list()[1:-1]:
        if int(i) >= int(value[0]) and int(i) <= int(value[1]):
            new_values_list_map.append(i)
            
    return MapGraph(map_df, map_df[new_values_list_map].sum(axis=1))

# ---------------------------------------------------------------
# Пользовательское обновление круговой диаграммы:
@app.callback(
    Output(component_id='pie', component_property='figure'),
    [Input(component_id='my-range-slider1', component_property='value')]
)

def PieChart_upd(value):
#     del temp_df_pie
    new_values_list_map = []
    for i in distr_df.columns.to_list()[1::]:
        if int(i) >= int(value[0]) and int(i) <= int(value[1]):
            new_values_list_map.append(i)

    temp_df_pie = pd.DataFrame({'distributors': distr_df['distributors'].to_list(), 
                                'sum': distr_df[new_values_list_map].sum(axis=1)})    

    temp_df_pie = temp_df_pie.sort_values(by=['sum'], ascending=False, ignore_index=True).loc[:9]
    temp_df_pie = temp_df_pie.append({'distributors': 'Others', 'sum': int(sum(distr_df[new_values_list_map].sum(axis=1).to_list())) - int(sum(temp_df_pie['sum'].to_list()))},
                                     ignore_index=True)

    temp_df_pie['colors'] = get_color("peach", np.linspace(0, 1, temp_df_pie.shape[0]))
    
    return PieChart(temp_df_pie)

# ---------------------------------------------------------------
# Пользовательское обновление диаграммы рассеивания:
@app.callback(
    Output(component_id='fig', component_property='figure'),
    [Input(component_id='my_dropdown', component_property='value')]
)

def ScatterPlotGraph_upd(column_chosen):
    conn_data = connect(host="localhost",
                        port='3306',
                        user='root',
                        password='Kinopoisk231.',
                        database='kinopoisk_movies')
    cur_data = conn_data.cursor()
    scatter_plot_df_new = pd.read_sql('''SELECT name, budget_USD_d, ratingKinopoisk, rusBO_USD_d, genres 
                                         FROM kinopoisk_movies.movies_upd mu
                                         WHERE genres LIKE "%{}%"'''.format(column_chosen), 
                                      con=conn_data)
    return ScatterPlotGraph(scatter_plot_df_new)

# ---------------------------------------------------------------
# Пользовательское обновление диаграммы распределения:
@app.callback(
    Output(component_id='fig1', component_property='figure'),
    [Input(component_id='my_dropdown1', component_property='value')]
)

def DistplotGraph_upd(column_chosen1):
    conn_data = connect(host="localhost",
                        port='3306',
                        user='root',
                        password='Kinopoisk231.',
                        database='kinopoisk_movies')
    cur_data = conn_data.cursor()
    distplot_plot_df_new = pd.read_sql('''SELECT name, worldBO_USD_d - budget_USD_d AS revenue, ratingAgeLimits
                                          FROM kinopoisk_movies.movies_upd mu
                                          WHERE ratingAgeLimits REGEXP "{}"'''.format('|'.join([str(i) for i in column_chosen1])),
                                       con=conn_data)
    return DistplotGraph(distplot_plot_df_new)

# ---------------------------------------------------------------
# Запуск приложения:
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
